<a href="https://colab.research.google.com/github/MakotoEndo8510/Qutip_expl/blob/main/visualization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Block Spehere animation